In [1]:
# ------------------------ util --------------------------------- #
import torch
from torch.autograd import Variable

def accuracy(y_pred, y):
    return (torch.max(y_pred, 1)[1] == y).float().mean().item()

''' Returns the mean accuracy on the test set, given a model '''
def eval_on_test(test_dataloader, model_fn):
    acc = 0
    for x, y in test_dataloader:
        x, y = Variable(x), Variable(y)

        if torch.cuda.is_available():
            x, y = x.cuda(), y.cuda()
        acc += accuracy(model_fn(x), y)
    return round(acc / float(len(test_dataloader)), 3)

''' Converts a list of (x, x) pairs into two Tensors ''' 
def into_tensor(data, into_vars=True):
    X1 = [x[0] for x in data]
    X2 = [x[1] for x in data]
    if torch.cuda.is_available():
        return Variable(torch.stack(X1)).cuda(), Variable(torch.stack(X2)).cuda()
    return Variable(torch.stack(X1)), Variable(torch.stack(X2))

In [2]:
# ------------------------- data -------------------------------- #
import torchvision
from torchvision import transforms
import torch
import numpy as np
import random
from torch.utils.data import DataLoader, Dataset
import pickle, gzip

class USPS(Dataset): # path -> where the data is
    def __init__(self, path='/content/drive/My Drive/DS2/usps_28x28.pkl', transform=None, train=True, all=False):
        self.train = train
        self.transform = transform
        x_train, y_train, x_test, y_test = [], [], [], []
        with gzip.open(path, 'rb') as f:
             (x_train, y_train), (x_test, y_test) = pickle.load(f, encoding='bytes')
        x = np.concatenate([x_train, x_test])
        y = np.concatenate([y_train, y_test], axis=0)
        self.data = x
        self.labels = y
        self.train_imgs = x_train
        self.train_labels = y_train
        self.test_imgs = x_test
        self.test_labels = y_test
        self.len_train = len(y_train)
        self.len_test = len(y_test)
    
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self, i):
        if all: img, label = self.data[i], self.labels[i]
        elif not all and self.train: img, label = self.train_imgs[i], self.train_labels[i]
        elif not all and not self.train: img, label = self.test_imgs[i], self.test_labels[i]
        if self.transform: img = self.transform(img)
        return img, label
# Returns dataloader or the dataset
def usps_dataloader(batch_size=256, train=True, cuda=False, n=1800, all=True, return_loader=False):
    trans = transforms.Compose([
        transforms.ToTensor(),
        transforms.ToPILImage(),
        transforms.Grayscale(),
        transforms.Resize(size=(16,16)),
        transforms.ToTensor(),
    ])
    dataset = USPS(transform=trans, train=train, all=all)  
    if all and return_loader: return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    if not all and return_loader: return DataLoader(dataset=dataset[:n], batch_size=batch_size, shuffle=True)
    if not all:
        X = torch.FloatTensor(n, 1, 16, 16)
        Y = torch.LongTensor(n)
        inds = torch.randperm(len(dataset))[:n]
    else:
        X = torch.FloatTensor(len(dataset), 1, 16, 16)
        Y = torch.LongTensor(len(dataset))
        inds = torch.randperm(len(dataset))
    for i, index in enumerate(inds):
        x, y = dataset[index]
        X[i] = x
        Y[i] = y
    return X, Y


''' Returns the MNIST dataloader '''
def mnist_dataloader(batch_size=256, train=True, cuda=False):
    trans = transforms.Compose([
        transforms.Resize(size=(16,16)),
        transforms.ToTensor()
    ])
    dataset = torchvision.datasets.MNIST('/content/drive/My Drive/ds2/data', download=True, train=train, transform=trans)
    return torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=2, pin_memory=cuda)


''' Returns the SVHN dataloader '''
def svhn_dataloader(batch_size=256, train=True, cuda=False):
    
    transform = torchvision.transforms.Compose([
        torchvision.transforms.Resize((16, 16)),
        torchvision.transforms.Grayscale(),
        torchvision.transforms.ToTensor()
    ])
    dataset = torchvision.datasets.SVHN('/content/drive/My Drive/ds2/data', download=True, split=('train' if train else 'test'), transform=transform)
    return torch.utils.data.DataLoader(dataset, shuffle=True, batch_size=batch_size, num_workers=2, pin_memory=cuda)


''' Samples a subset from source into memory '''
def sample_data(n=2000, all=True, data='mnist'):
    if data == 'mnist':
        transf = torchvision.transforms.Compose([
            torchvision.transforms.Resize(size=(16,16)),
            torchvision.transforms.ToTensor()
        ])
        dataset = torchvision.datasets.MNIST('/content/drive/My Drive/ds2/data', download=True, train=True, transform=transf)
    elif data == 'svhn': 
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((16, 16)),
            torchvision.transforms.Grayscale(),
            torchvision.transforms.ToTensor()
        ])
        dataset = torchvision.datasets.SVHN('/content/drive/My Drive/ds2/data', download=True, split=('train' if train else 'test'), transform=transform)
    #elif data == 'usps':
    if all: 
        inds = torch.randperm(len(dataset))
        X = torch.FloatTensor(len(dataset), 1, 16, 16)
        Y = torch.LongTensor(len(dataset))
    if not all:
        X = torch.FloatTensor(n, 1, 16, 16)
        Y = torch.LongTensor(n)
        inds = torch.randperm(len(dataset))[:n]
    for i, index in enumerate(inds):
        x, y = dataset[index]
        X[i] = x
        Y[i] = y
    return X, Y

    #create a sample data for others experiments (3 to 6)


''' Returns a subset of the target domain such that it has n_target_samples per class '''
def create_target_samples(n=1, target='svhn'):
    if target == 'svhn':
        transform = torchvision.transforms.Compose([
            torchvision.transforms.Resize((16, 16)),
            torchvision.transforms.Grayscale(),
            torchvision.transforms.ToTensor()
        ])
        dataset = torchvision.datasets.SVHN('/content/drive/My Drive/ds2/data', download=True, split='train', transform=transform)
    elif target == 'mnist':
        transf = torchvision.transforms.Compose([
            torchvision.transforms.Resize(size=(16,16)),
            torchvision.transforms.ToTensor()
        ])
        dataset = torchvision.datasets.MNIST('/content/drive/My Drive/ds2/data', download=True, train=True, transform=transf)
    elif target == 'usps':
        trans = transforms.Compose([
            transforms.ToTensor(),
            transforms.ToPILImage(),
            transforms.Grayscale(),
            transforms.Resize(size=(16,16)),
            transforms.ToTensor(),
        ])
        dataset = USPS(transform=trans, train=True, all=False)
    print(target, len(dataset), len(dataset[0]))
    X, Y = [], []
    classes = 10 * [n]

    i = 0
    while True:
        if len(X) == n*10:
            break
        x, y = dataset[i]
        if classes[y] > 0:
            X.append(x)
            Y.append(y)
            classes[y] -= 1
        i += 1

    assert(len(X) == n*10)
    return torch.stack(X), torch.from_numpy(np.array(Y))

''' 
    Samples uniformly groups G1 and G3 from D_s x D_s and groups G2 and G4 from D_s x D_t  
'''
def create_groups(X_s, Y_s, X_t, Y_t, seed=1):
    #change seed so every time wo get group data will different in source domain,but in target domain, data not change
    torch.manual_seed(1 + seed)
    torch.cuda.manual_seed(1 + seed)
    n=X_t.shape[0] #10*shot
    #shuffle order
    classes = torch.unique(Y_t)
    shuf_classes=classes[torch.randperm(len(classes))]
    
    class_num=shuf_classes.shape[0]
    shot=n//class_num
    #shot = n
    
    def get_idx(map_labels, labels):
        for i in range(len(labels)): map_labels[labels[i].item()].append(i)
        return map_labels
    def g1_sample(Y, ids, X): # X^s_i, X^s_s
        i1 = random.choice(ids)
        i2 = random.choice(ids)
        return (X[i1], X[i2]), (Y[i1], Y[i2])
    def g2_sample(Ys, Yt, idx_s, idx_t, Xs, Xt): # X^s_i, X^t_i
        s = random.choice(idx_s)
        t = random.choice(idx_t)
        return ( Xs[s], Xt[t] ), (Ys[s], Yt[t])
    def g3_sample(X, Y, idx_i, idx_j): # X^s_i , X^s_j
        i = random.choice(idx_i)
        j = random.choice(idx_j)
        return (X[i], X[j]), (Y[i], Y[j])
    def g4_sample(Xs, Xt, Ys, Yt, idx_s, idx_t): # X^s_i, X^t_j
        i = random.choice(idx_s)
        j = random.choice(idx_t)
        return (Xs[i], Xt[j]), (Ys[i], Yt[j])
    
    s_idx = torch.arange(0, len(X_s))
    t_idx = torch.arange(0, len(X_t))
    s_labels = np.unique(Y_s)
    t_labels = np.unique(Y_t)
    s_map = dict(zip(s_labels, [ [] for i in range(len(s_labels)) ]))
    t_map = dict(zip(t_labels, [ [] for i in range(len(t_labels)) ]))
    
    s_map = get_idx(s_map, Y_s)
    t_map = get_idx(t_map, Y_t)
    G1, G2, G3, G4 = [], [], [], []
    Y1, Y2, Y3, Y4 = [], [], [], []
    for i in s_labels:
        for j in t_labels:
            tmp1 = g1_sample(Y_s, s_map[i], X_s)
            tmp2 = g2_sample(Y_s, Y_t, s_map[i], t_map[j], X_s, X_t)
            ii = i
            while ii == j or ii not in s_labels: ii = random.choice(s_labels)
            tmp3 = g3_sample(X_s, Y_s, s_map[ii], s_map[j])
            ii = i
            while ii == j or ii not in s_labels: ii = random.choice(s_labels)
            tmp4 = g4_sample(X_s, X_t, Y_s, Y_t, s_map[ii], t_map[j])
            G1.append(tmp1[0])
            G2.append(tmp2[0])
            G3.append(tmp3[0])
            G4.append(tmp4[0])
            Y1.append(tmp1[1])
            Y2.append(tmp2[1])
            Y3.append(tmp3[1])
            Y4.append(tmp4[1])
    groups = [G1, G2, G3, G4]
    labels = [Y1, Y2, Y3, Y4]
    print(n)
    # Make sure we sampled enough samples
    for g in groups:
        #print(len(g), n)
        assert(len(g) > n)
    return groups, labels

''' Sample groups G1, G2, G3, G4 '''
def sample_groups(n_target_samples=2, train_mode=2, n=2000, source='mnist', target='svhn'):
    if training_mode == 2:
        if source == 'usps': X_s, Y_s = usps_dataloader(all=False, train=True, return_loader=False)
        else: X_s, y_s = sample_data(all=True, data=source)
    else:
        if source == 'usps': X_s, Y_s = usps_dataloader(all=True, return_loader=False, n=n)
        else: X_s, y_s = sample_data(n=n, all=False, data=source)
    X_t, y_t = create_target_samples(n=n_target_samples, target=target)
    
    #print("Sampling groups")
    return create_groups(X_s, y_s, X_t, y_t), (X_s, y_s, X_t, y_t)

In [3]:
# ---------------------------------- models ---------------------------------- #
import torch
import torch.nn.functional as F
from torch import nn

'''
    Domain-Class Discriminator (see (3) in the paper)
    Takes in the concatenated latent representation of two samples from
    G1, G2, G3 or G4, and outputs a class label, one of [0, 1, 2, 3]
'''
class DCD(nn.Module):
    def __init__(self, H=64, D_in=256):
        super(DCD, self).__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.fc2 = nn.Linear(H, H)
        self.out = nn.Linear(H, 4)

    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(out)
        return F.softmax(self.out(out), dim=1)

''' Called h in the paper. Gives class predictions based on the latent representation '''
class Classifier(nn.Module):
    def __init__(self, D_in=64):
        super(Classifier, self).__init__()
        self.out = nn.Linear(D_in, 10)

    def forward(self, x):
        return F.softmax(self.out(x), dim=1)

''' 
    Creates latent representation based on data. Called g in the paper.
    Like in the paper, we use g_s = g_t = g, that is, we share weights between target
    and source representations.
    Model is as specified in section 4.1. See https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py
'''
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
    
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1   = nn.Linear(16, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 64)
        self.flat  = nn.Flatten()

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = self.flat(out)
        #out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

In [4]:
# ---------------------------- training -------------------------------------- #

import torch
from torch import nn
from torch import optim
from torch.autograd import Variable
#from data import mnist_dataloader, svhn_dataloader
#from models import Classifier, Encoder, DCD
#from util import eval_on_test, into_tensor
import random
import matplotlib.pyplot as plt

def model_fn(encoder, classifier):
    return lambda x: classifier(encoder(x))

''' Pretrain the encoder and classifier as in (a) in figure 2. '''
def pretrain(epochs=5, batch_size=256, cuda=False, domain='svhn'):
    print("Test: ", domain)
    if domain == 'svhn':
        train_loader = svhn_dataloader(train=True, cuda=cuda)
        test_loader = svhn_dataloader(train=False, cuda=cuda)
    elif domain == 'mnist':
        train_loader = mnist_dataloader(train=True, cuda=cuda)
        test_loader = mnist_dataloader(train=False, cuda=cuda)
    elif domain == 'usps':
        train_loader = usps_dataloader(train=True, cuda=cuda)
        test_loader = usps_dataloader(train=False, cuda=cuda)

    X_s, y_s, _, _ = data

    classifier = Classifier()
    encoder = Encoder()

    if cuda:
        classifier.cuda()
        encoder.cuda()
        device = 'cuda'
    else: device = 'cpu'

    ''' Jointly optimize both encoder and classifier ''' 
    optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()))
    loss_fn = nn.CrossEntropyLoss()
    print("A entrenar ... ", epochs)

    for epoch in range(epochs):
        classifier.train()
        encoder.train()
        train_loss = 0
        valid_loss = 0
        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            y_pred = classifier(encoder(imgs))
            loss = loss_fn(y_pred, labels)
            train_loss += loss.item()
            loss.backward()
            optimizer.step()
        acc = 0
        classifier.eval()
        encoder.eval()
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            y_pred = classifier(encoder(imgs))
            loss = loss_fn(y_pred, labels)
            valid_loss += loss.item()
            acc+=(torch.max(y_pred,1)[1]==labels).float().mean().item()
        accuracy = round( acc / float(len(test_loader)), 3 )
        #print( "STEP1 ------- Epoch {} / {} -> \t Accuracy: {} \t Train loss: {} \t Validation loss: {}".format(epoch, epochs, accuracy, train_loss/len(train_loader), valid_loss/len(test_loader)) )
    
    return encoder, classifier

''' Train the discriminator while the encoder is frozen '''
def train_discriminator(encoder, groups, n_target_samples=2, cuda=False, epochs=20, source='mnist', target='svhn', training_mode=2):
    if training_mode == 2: x_s , Y_s = sample_data(all=True, data=source)
    else: X_s, Y_s = sample_data( n = (2000 if source=='mnist' else 1800), all=False, data=source )
    X_t, Y_t = create_target_samples(n=n_target_samples, target=target)

    if cuda: device = 'cuda'
    else: device = 'cpu'

    discriminator = DCD(H=512, D_in=128) # Takes in concatenated hidden representations
    discriminator.to(device)
    encoder.to(device)
    loss_fn = nn.CrossEntropyLoss()
    #class_optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()) )
    # Only train DCD
    optimizer = optim.Adam(discriminator.parameters())
    
    # Size of group G2, the smallest one, times the amount of groups
    n_iters = 4 * n_target_samples
    #print("\t\t --- Training DC ---")
    
    encoder.eval()
    discriminator.train()
    loss_mean = []
    for epoch in range(epochs):
        acc = 0
        for ii in range(n_iters):
            optimizer.zero_grad()
            i = ii if(ii<4) else ii%4
            x1, x2 = groups[i][ random.randint(0, len(groups[i])-1) ]
            x1, x2 = Variable(x1), Variable(x2)
            truth = torch.tensor([i])
            x1, x2 = x1.to(device), x2.to(device)
            z_1 = encoder(x1.unsqueeze(0)) # z_i : latent sppace of x1
            z_2 = encoder(x2.unsqueeze(0))
            z_cat = torch.cat([z_1, z_2], dim=1)
            z_cat = z_cat.to(device)
            y_pred = discriminator(z_cat)
            loss = loss_fn(y_pred, truth)
            loss_mean.append(loss.item())
            loss.backward()
            optimizer.step()


        #print("STEP 2 ------- Epoch: {} / {}  ->  Loss: {}".format(epoch, epochs, loss_mean[-1]/n_iters ) )    
  
    return discriminator

''' FADA Loss, as given by (4) in the paper. The minus sign is shifted because it seems to be wrong '''
def fada_loss(y_pred_g2, g1_true, y_pred_g4, g3_true, gamma=0.2):
    return -gamma * torch.mean(g1_true * torch.log(y_pred_g2) + g3_true * torch.log(y_pred_g4))

''' Step three of the algorithm, train everything except the DCD '''
def train(encoder, discriminator, classifier, data, groups, groups_labels, n_target_samples=2, cuda=False, epochs=20, batch_size=256, plot_accuracy=False, target='svhn', source='mnist', test_n=None):
    if target == 'mnist': test_loader = mnist_dataloader(train=False, cuda=cuda)
    elif target == 'svhn': test_loader = svhn_dataloader(train=False, cuda=cuda)
    elif target == 'usps':
        if test_n: test_loader = usps_dataloader(train=False, cuda=cuda, n=test_n, return_loader=True)
        else: test_loader = usps_dataloader(train=False, cuda=cuda, return_loader=True)
    
    if cuda: device = 'cuda'
    else: device = 'cpu'
    encoder.to(device)
    classifier.to(device)
    discriminator.to(device)

    X_s, Y_s, X_t, Y_t = data

    G1, G2, G3, G4 = groups
    Y1, Y2, Y3, Y4 = groups_labels

    ''' Two optimizers, one for DCD (which is frozen) and one for class training ''' 
    class_optimizer = optim.Adam(list(encoder.parameters()) + list(classifier.parameters()))
    #class_optimizer = optim.Adam(classifier.parameters())
    encoder_optimizer = optim.Adam(encoder.parameters())
    dcd_optimizer = optim.Adam(discriminator.parameters())

    loss_fn = nn.CrossEntropyLoss()
    loss_dcd = nn.CrossEntropyLoss()
    n_iters = 4 * n_target_samples
    
    losses = []
    accuracies = []
    for epoch in range(epochs):
        # Training g & h
        # Set g & h training mode while DCD is frozen
        encoder.train()
        classifier.train()
        discriminator.eval()
        epoch_loss = 0
        loss_mean = []
        gh_losses = []
        dcd_losses = []
        aux_loss = 0
        for i in range(n_target_samples): # n-shot -> n labels 
            for j in range(4): # 4 groups
            # Select samples from Gj
                class_optimizer.zero_grad()
                x1, x2 = groups[j][i]
                y1, y2 = groups_labels[j][i]
                x1, y1 = x1.type(torch.FloatTensor), y1.type(torch.LongTensor)
                x1, y1 = x1.to(device), y1.unsqueeze(0).to(device)
                z1 = encoder(x1.unsqueeze(0))
                z1 = z1.to(device)
                y_pred1 = classifier(z1)
                loss_1 = loss_fn(y_pred1, y1)
                aux_loss += loss_1.item()
                loss_1.backward()
                class_optimizer.step()

                class_optimizer.zero_grad()
                x2, y2 = x2.type(torch.FloatTensor), y2.type(torch.LongTensor)
                x2, y2 = x2.to(device), y2.unsqueeze(0).to(device)
                z2 = encoder(x2.unsqueeze(0))
                z2 = z2.to(device)
                y_pred2 = classifier(z2)
                loss_2 = loss_fn(y_pred2, y2)
                aux_loss += loss_2.item()
                loss_2.backward()
                class_optimizer.step()
                if j == 1 or j == 3:
                    encoder_optimizer.zero_grad()
                    x_1, x_2 = groups[j][i]
                    y_1, y_2 = groups_labels[j][i]
                    x_1, y_1 = x_1.type(torch.FloatTensor), y_1.type(torch.LongTensor)
                    x_2, y_2 = x_2.type(torch.FloatTensor), y_2.type(torch.LongTensor)
                    x1, x2, y1, y2 = x_1.to(device), x_2.to(device), y_1.unsqueeze(0).to(device), y_2.unsqueeze(0).to(device)
                    z1 = encoder(x1.unsqueeze(0))
                    z2 = encoder(x2.unsqueeze(0))
                    z1, z2 = z1.to(device), z2.to(device)
                    zcat = torch.cat([z1, z2], dim=1).to(device)
                    yd = discriminator(zcat)
                    loss_d = loss_fn(yd, torch.tensor([j-1]).to(device) )
                    aux_loss += loss_d.item()
                    loss_d.backward()
                    encoder_optimizer.step()
        gh_losses.append(aux_loss/(6*n_target_samples))

        # Train DCD while g&h are frozen
        encoder.eval()
        classifier.eval()
        discriminator.train()
        aux_loss = 0
        for i in range(n_target_samples): # n-shots
            for j in range(4): # Group
                dcd_optimizer.zero_grad()
                x1, x2 = groups[j][i]
                x1, x2 = x1.type(torch.FloatTensor), x2.type(torch.FloatTensor)
                truth = torch.tensor([j]).type(torch.LongTensor)
                x1, x2 = Variable(x1), Variable(x2)
                x1, x2 = x1.to(device), x2.to(device)
                z1 = encoder(x1.unsqueeze(0))
                z2 = encoder(x2.unsqueeze(0))
                z_cat = torch.cat([z1,z2], dim=1).to(device)
                g_pred = discriminator(z_cat)
                lossd = loss_dcd(g_pred.to(device), truth.to(device))
                aux_loss += lossd.item()
                lossd.backward()
                dcd_optimizer.step()
        dcd_losses.append(aux_loss/(4*n_target_samples))
    
        # Testing for accuracy
        classifier.eval()
        encoder.eval()
        discriminator.eval()
        acc = 0
        aux = 0
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            y_pred = classifier(encoder(imgs))
            aux = (torch.max(y_pred,1)[1]==labels).float().mean().item()
            #print("aux: ", aux, end="\t")
            acc += aux
        accuracies.append(acc/len(test_loader))

        #print("STEP 3/4: Epoch: {} / {} -> \t g & h losses: {} \t DCD loss: {} \t Accuracy: {}".format(epoch, epochs, gh_losses[-1], dcd_losses[-1], accuracies[-1]) )

        #acc = eval_on_test(test_dataloader, model_fn(encoder, classifier))
        #if plot_accuracy:
            #accuracies.append(acc)
        # Train DCD
    print("{} to {} \t {}-shot \t {} epochs".format(source.upper(), target.upper(), n_target_samples, epochs) )
    if plot_accuracy:
        plt.plot(range(len(accuracies)), accuracies)
        plt.title(target.upper()+" test accuracy")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.show()
    print("max accuracy: {} - mean accuracy: {} - min accuracy: {}".format(np.max(accuracies), np.mean(accuracies), np.min(accuracies)))
    return np.max(accuracies), np.mean(accuracies), np.min(accuracies)

In [ ]:
# ------------------------------ main ---------------------------------------- #
#from training import pretrain, train_discriminator, train
#from data import sample_groups
import torch

n_target_samples_init = 1
plot_accuracy = True
training_mode = 1
"""
training_mode = 1 -> take 2000 train samples from mnist and 1800 train samples of usps
training_mode = 2 -> take all train samples and all test samples (for testing)
"""
data_options = ['mnist', 'svhn', 'usps']
source = data_options[0] # M -> U
target = data_options[2] 
source_samples = 2000
target_samples = 1800
max_accs, mean_accs, min_accs = [], [], []
if __name__ == '__main__':
    file = open('/content/drive/My Drive/DS2/resultados.txt', 'a')
    cuda = torch.cuda.is_available()
    for n_target_samples in range(n_target_samples_init, 3, 1):
        max_aux , mean_aux, min_aux = [], [], []
        for k in range(3):
            print("Repetition: {} \t task: {} to {} \t training mode: {} \t {}-shot \t source samples: {} \t target samples: {}".format(k, source, target, training_mode, n_target_samples, source_samples, target_samples) )
            if training_mode == 2: groups, data = sample_groups(n_target_samples=n_target_samples, train_mode=training_mode, source=source, target=target)
            if training_mode == 1: groups, data = sample_groups(n_target_samples=n_target_samples, train_mode=training_mode, source=source, target=target, n=source_samples)  # M->U or U->M
            labels = groups[1]
            groups = groups[0]
            encoder, classifier = pretrain(cuda=cuda, epochs=20, domain=source) # Train g and h on D_s ; epoch=20
            discriminator = train_discriminator(encoder, groups, n_target_samples, False, epochs=50, source=source, target=target, training_mode=training_mode) # Train DCD sharing weights g_s = g_t = g ; epochs=50
            max_ac, mean_ac, min_ac = train(encoder=encoder, discriminator=discriminator, classifier=classifier, data=data, groups=groups, groups_labels=labels, n_target_samples=n_target_samples, cuda=cuda, epochs=200, plot_accuracy=plot_accuracy, target=target, source=source, test_n=target_samples)
            del encoder
            del classifier
            del discriminator
            max_aux.append(max_ac)
            mean_aux.append(mean_ac)
            min_aux.append(min_ac)
        max_accs.append(max_aux)
        mean_accs.append(mean_aux)
        min_accs.append(min_aux)
        file.write( "\nTask: {} to {} \t epochs: 300 \t {}-shot\n".format(source, target, n_target_samples) )
        file.write( "All of max accuracys: {} \t mean of max accs: {}\n".format(max_aux, np.mean(max_aux)) )
        file.write( "All of mean accuracys: {} \t mean of mean accs: {}\n".format(mean_aux, np.mean(mean_aux)) )
        file.write( "All of min accuracys: {} \t mean of min accs: {}\n\n".format(min_aux, np.mean(min_aux)) )
        print( "All of max accuracys: {} \t mean of max accs: {}".format(max_aux, np.mean(max_aux)) )
        print( "All of mean accuracys: {} \t mean of mean accs: {}".format(mean_aux, np.mean(mean_aux)) )
        print( "All of min accuracys: {} \t mean of min accs: {}\n".format(min_aux, np.mean(min_aux)) )
    file.close()

Repetition: 0 	 task: mnist to usps 	 training mode: 1 	 1-shot 	 source samples: 2000 	 target samples: 1800
usps 9298 2
10
Test:  mnist
A entrenar ...  20
